# Most frequent words

In [27]:
import pandas as pd
from sklearn.cluster import KMeans
import nltk
from nltk.corpus import stopwords
import numpy as np
from pattern.text.es import singularize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
from share import select_secondary_keywords_ngrams
from tqdm.notebook import tqdm
tqdm.pandas()

In [14]:
all_df = pd.read_csv('all_df.csv')
all_df.shape

(352834, 8)

In [21]:
def split_content(content):
    if not content:
        return []
    return ' '.join(re.split(r"\s*[:?!;|,-.]\s*", content))

In [22]:
all_df["preprocess"] = all_df.progress_apply(lambda row: split_content(row.Texto), axis=1)
all_df

  0%|          | 0/352834 [00:00<?, ?it/s]

,Unnamed: 0,Participante,Texto,Sentimiento,Palabras,Dia,Mes,Anio,preprocess
0,0,-PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,Informarles que el día de hoy voy a enviar al...,0.001959,58,6,12,2018,Informarles que el día de hoy voy a enviar al...
1,1,-PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,Les doy a conocer los nombres de quienes están...,0.771382,10,6,12,2018,Les doy a conocer los nombres de quienes están...
2,2,-PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,La abogada Loretta Ortiz. Ella es egresada de ...,0.034682,27,6,12,2018,La abogada Loretta Ortiz Ella es egresada de l...
3,3,-PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,"También, la licenciada Celia Maya García. Ella...",0.003624,46,6,12,2018,También la licenciada Celia Maya García Ella e...
4,4,-PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,Y el tercer abogado propuesto es el licenciado...,0.017006,39,6,12,2018,Y el tercer abogado propuesto es el licenciado...
...,...,...,...,...,...,...,...,...,...
352829,352829,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,Y va a seguir ayudando Ricardo en Coahuila par...,0.000000,38,15,12,2022,Y va a seguir ayudando Ricardo en Coahuila par...
352830,352830,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,Esto me lo plantearon en una reunión que tuve ...,0.000000,67,15,12,2022,Esto me lo plantearon en una reunión que tuve ...
352831,352831,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,"Entonces, él va a seguir ayudándonos en esto.",0.000000,8,15,12,2022,Entonces él va a seguir ayudándonos en esto
352832,352832,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,"Y con mucha responsabilidad, con mucha madurez...",0.000000,44,15,12,2022,Y con mucha responsabilidad con mucha madurez ...


In [23]:
all_df.to_csv('/Users/almacuevas/work_projects/conferencias_matutinas_amlo/all_df_preprocess.csv')

In [28]:
df_year = all_df.loc[all_df["Anio"]==2018][0:100]
frequent_words = select_secondary_keywords_ngrams(df_year.preprocess, 5, 5)  #[0:max_n]
frequent_words

,term,rank
0,sara pablo radio fórmula presidente,1.000000
1,eso puede seguir pasando país,1.000000
2,jorge monroy periódico el economista,1.000000
3,nosotros vamos respaldar compromisos inversion...,1.000000
4,en días creo va comenzar,1.000000
...,...,...
935,adulto mayor ciudad méxico muchas,0.188982
936,méxico muchas acciones eso general,0.188982
937,financiar programa bienestar contratación jóvenes,0.188982
938,estudiantes niveles escolaridad aumenta pensión,0.188982


# Unsupervised classification of all conversations

In [25]:
vectorizer = TfidfVectorizer(stop_words=stopwords.words("spanish"))
X = vectorizer.fit_transform(frequent_words["term"])
true_k = 4 # Number of groups
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind])

Cluster 0:
 ser
 gente
 va
 puede
 van
 debe
 mucha
 vamos
 entonces
 humilde
Cluster 1:
 si
 usted
 sé
 va
 ver
 saber
 puede
 preguntarle
 presidente
 gobierno
Cluster 2:
 dinero
 quiero
 presupuesto
 lavado
 decir
 va
 pueblo
 hacer
 gobierno
 si
Cluster 3:
 va
 vamos
 mil
 méxico
 gobierno
 presidente
 entonces
 país
 ahora
 van
